In [2]:
import sys
import numpy as np
import pandas as pd
import os 
import gc
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import StratifiedKFold, KFold
#from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error as mse  # 评测标准
from sklearn.preprocessing import LabelEncoder
import datetime
import time
import lightgbm as lgb
import xgboost as xgb
import warnings

In [3]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [4]:
path  = './input/'
train_sales  = pd.read_csv(path+'train_sales_data.csv')  #  销量训练集 <-标签
train_search = pd.read_csv(path+'train_search_data.csv') # 搜索量训练集
train_user   = pd.read_csv(path+'train_user_reply_data.csv') # 评论训练集
evaluation_public = pd.read_csv(path+'evaluation_public.csv') # 测试集
submit_example    = pd.read_csv(path+'submit_example.csv') # 样例集
data = pd.concat([train_sales, evaluation_public], ignore_index=True) # 将销量训练集与预测集连接
# print(data)
# data.to_csv('./data2.csv',index=False, encoding='gb2312')
data = data.merge(train_search, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
data = data.merge(train_user, 'left', on=['model', 'regYear', 'regMonth']) # 左连接，将该车型相同时间的其他特征连接起来

data['label'] = data['salesVolume'] # <-标签列
data['id'] = data['id'].fillna(0).astype(int) # 空值 用0填充
# map映射   
# drop_duplicates 对model(SUV（车型编码）-1车身类型[最细致的一个特征])去重，去重后便于map标号

data['bodyType'] = data['model'].map(train_sales.drop_duplicates('model').set_index('model')['bodyType'])
#data['bodyType'].to_csv('./data3.csv',index=False, encoding='gb2312')

#LabelEncoder
for i in ['bodyType', 'model']:
    data[i] = data[i].map(dict(zip(data[i].unique(), range(data[i].nunique()))))
data['mt'] = (data['regYear'] - 2016) * 12 + data['regMonth']
data.to_csv('./data4.csv',index=False, encoding='gb2312')

In [5]:
def get_stat_feature(df_):   
    df = df_.copy()
    stat_feat = []
    df['model_adcode'] = df['adcode'] + df['model']
    df['model_adcode_mt'] = df['model_adcode'] * 100 + df['mt']
    for col in tqdm(['label','popularity']):
        # shift
        for i in [1,2,3,4,5,6,7,8,9,10,11,12]:
            stat_feat.append('shift_model_adcode_mt_{}_{}'.format(col,i))
            df['model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'] + i
            df_last = df[~df[col].isnull()].set_index('model_adcode_mt_{}_{}'.format(col,i))
            df['shift_model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'].map(df_last[col])    
    return df,stat_feat

In [6]:
def score(data, pred='pred_label', label='label', group='model'):
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    data_agg = data.groupby('model').agg({
        pred:  list,
        label: [list, 'mean']
    }).reset_index()
    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)	

In [23]:
def get_model_type(train_x,train_y,valid_x,valid_y,m_type='lgb'):   
    if m_type == 'lgb':
        model = lgb.LGBMRegressor(
                                num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='mse',
                                max_depth=-1, learning_rate=0.05, min_child_samples=5, random_state=2019,
                                n_estimators=2000, subsample=0.9, colsample_bytree=0.7,
                                )
        model.fit(train_x, train_y, 
              eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              categorical_feature=cate_feat, 
              early_stopping_rounds=100, verbose=100)      
    elif m_type == 'xgb':
        model = xgb.XGBRegressor(
                                max_depth=5 , learning_rate=0.05, n_estimators=2000, 
                                objective='reg:gamma', tree_method = 'hist',subsample=0.9, 
                                colsample_bytree=0.7, min_child_samples=5,eval_metric = 'rmse' 
                                )
        model.fit(train_x, train_y, 
              eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              early_stopping_rounds=100, verbose=100)   
    return model

In [24]:
def get_train_model(df_, m, m_type='lgb'):
    df = df_.copy()
    # 数据集划分
    st = 13
    all_idx   = (df['mt'].between(st , m-1))
    train_idx = (df['mt'].between(st , m-5))
    valid_idx = (df['mt'].between(m-4, m-4))
    test_idx  = (df['mt'].between(m  , m  ))
    print('all_idx  :',st ,m-1)
    print('train_idx:',st ,m-5)
    print('valid_idx:',m-4,m-4)
    print('test_idx :',m  ,m  )  
    # 最终确认
    train_x = df[train_idx][features]
    train_y = df[train_idx]['label']
    valid_x = df[valid_idx][features]
    valid_y = df[valid_idx]['label']   
    # get model
    model = get_model_type(train_x,train_y,valid_x,valid_y,m_type)  
    # offline
    df['pred_label'] = model.predict(df[features])
    best_score = score(df[valid_idx]) 
    # online
    if m_type == 'lgb':
        model.n_estimators = model.best_iteration_ + 100
        model.fit(df[all_idx][features], df[all_idx]['label'], categorical_feature=cate_feat)
    elif m_type == 'xgb':
        model.n_estimators = model.best_iteration + 100
        model.fit(df[all_idx][features], df[all_idx]['label'])
    df['forecastVolum'] = model.predict(df[features]) 
    print('valid mean:',df[valid_idx]['pred_label'].mean())
    print('true  mean:',df[valid_idx]['label'].mean())
    print('test  mean:',df[test_idx]['forecastVolum'].mean())
    # 阶段结果
    sub = df[test_idx][['id']]
    sub['forecastVolum'] = df[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)  
    return sub,df[valid_idx]['pred_label']

In [25]:
for month in [25,26,27,28]: 
    m_type = 'xgb' 
    
    data_df, stat_feat = get_stat_feature(data)
    
    num_feat = ['regYear'] + stat_feat
    cate_feat = ['adcode','bodyType','model','regMonth']
    
    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
    print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)   
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'label'      ] = sub['forecastVolum'].values	
sub_xgb = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub_xgb.columns = ['id','forecastVolum']
sub_xgb[['id','forecastVolum']].round().astype(int).to_csv('CCF_sales_xgb.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 28.05it/s]


29 29
all_idx  : 13 24
train_idx: 13 20
valid_idx: 21 21
test_idx : 25 25
[0]	validation_0-rmse:841.375	validation_1-rmse:1046.8
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:800.25	validation_1-rmse:1004.68
[199]	validation_0-rmse:260.992	validation_1-rmse:373.604
0.7060436029133734
valid mean: 543.93024
true  mean: 649.3121212121212
test  mean: 460.5905


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 31.33it/s]


29 29
all_idx  : 13 25
train_idx: 13 21
valid_idx: 22 22
test_idx : 26 26
[0]	validation_0-rmse:866.608	validation_1-rmse:1007.37
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:825.42	validation_1-rmse:966.248
[199]	validation_0-rmse:269.098	validation_1-rmse:403.121
0.5904416959454815
valid mean: 441.86932
true  mean: 616.5537878787878
test  mean: 328.8417


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 31.33it/s]


29 29
all_idx  : 13 26
train_idx: 13 22
valid_idx: 23 23
test_idx : 27 27
[0]	validation_0-rmse:881.697	validation_1-rmse:1071.89
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:840.504	validation_1-rmse:1029.19
[199]	validation_0-rmse:276.5	validation_1-rmse:389.132
0.6492073097142486
valid mean: 512.0814
true  mean: 673.0143939393939
test  mean: 391.85028


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 27.85it/s]


29 29
all_idx  : 13 27
train_idx: 13 23
valid_idx: 24 24
test_idx : 28 28
[0]	validation_0-rmse:900.647	validation_1-rmse:1451.3
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:859.319	validation_1-rmse:1408.35
[199]	validation_0-rmse:282.737	validation_1-rmse:801.228
0.40110547860321266
valid mean: 481.19583
true  mean: 899.8204545454546
test  mean: 382.5133


PermissionError: [Errno 13] Permission denied: 'CCF_sales_xgb.csv'

In [18]:
for month in [25,26,27,28]: 
    m_type = 'lgb' 
    
    data_df, stat_feat = get_stat_feature(data)
    
    num_feat = ['regYear'] + stat_feat
    cate_feat = ['adcode','bodyType','model','regMonth']
    
    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
    print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)   
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'label'      ] = sub['forecastVolum'].values	
sub_lgb = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub_lgb.columns = ['id','forecastVolum']
sub_lgb[['id','forecastVolum']].round().astype(int).to_csv('CCF_sales_lgb.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.15it/s]


29 29
all_idx  : 13 24
train_idx: 13 20
valid_idx: 21 21
test_idx : 25 25
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 8492.89	valid_1's l2: 35153.6
[200]	training's l2: 4546.93	valid_1's l2: 31997.7
[300]	training's l2: 2975.08	valid_1's l2: 31250.1
[400]	training's l2: 2097.97	valid_1's l2: 30863.9
[500]	training's l2: 1548.26	valid_1's l2: 30670.3
[600]	training's l2: 1186.17	valid_1's l2: 30450.2
[700]	training's l2: 941.965	valid_1's l2: 30312.3
[800]	training's l2: 752.927	valid_1's l2: 30220
[900]	training's l2: 614.003	valid_1's l2: 30146.6
[1000]	training's l2: 507.21	valid_1's l2: 30122.4
[1100]	training's l2: 417.38	valid_1's l2: 30072.3
[1200]	training's l2: 349.745	valid_1's l2: 30037.1
[1300]	training's l2: 293.352	valid_1's l2: 30013.1
[1400]	training's l2: 248.668	valid_1's l2: 30001.2
Early stopping, best iteration is:
[1379]	training's l2: 257.61	valid_1's l2: 29993.3
0.7595307137391416
valid mean: 597.5013335217342
true  mean: 6

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.57it/s]


29 29
all_idx  : 13 25
train_idx: 13 21
valid_idx: 22 22
test_idx : 26 26
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 9064.9	valid_1's l2: 42831.3
[200]	training's l2: 5011.29	valid_1's l2: 42129.5
[300]	training's l2: 3337	valid_1's l2: 41897.7
Early stopping, best iteration is:
[247]	training's l2: 4089.85	valid_1's l2: 41750.9
0.742167006506538
valid mean: 628.5670193841313
true  mean: 616.5537878787878
test  mean: 333.901352665761


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.34it/s]


29 29
all_idx  : 13 26
train_idx: 13 22
valid_idx: 23 23
test_idx : 27 27
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 10087	valid_1's l2: 33582
[200]	training's l2: 5480.63	valid_1's l2: 32030.1
Early stopping, best iteration is:
[186]	training's l2: 5790.86	valid_1's l2: 31914.4
0.7811053032455804
valid mean: 643.1109390693433
true  mean: 673.0143939393939
test  mean: 503.69364370375354


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.29it/s]


29 29
all_idx  : 13 27
train_idx: 13 23
valid_idx: 24 24
test_idx : 28 28
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 10693.1	valid_1's l2: 331691
[200]	training's l2: 6184.36	valid_1's l2: 325609
Early stopping, best iteration is:
[163]	training's l2: 7155.93	valid_1's l2: 324504
0.5975151216410015
valid mean: 644.2463500470923
true  mean: 899.8204545454546
test  mean: 500.87802249387806


In [22]:
sub_avg = pd.DataFrame()
sub_avg['id'] = sub_xgb['id']
sub_avg['forecastVolum'] = (sub_xgb['forecastVolum']+sub_lgb['forecastVolum'])/2
sub_avg[['id','forecastVolum']].round().astype(int).to_csv('CCF_sales_avg.csv', index=False)